In [1]:
import numpy as np
import tensorflow as tf
import core.preprocessing as pp
from core.net_models import FC, CNN
from keras.regularizers import l1, l2

LOGDIR = "./tmp/exo_test2/"

present = 3*12
future = 3*12

dm = pp.DataManager()
dm.load_all_and_split(kind='all')
dftrain = dm.detrended_train
dfdev = dm.detrended_dev

scale = pp.CenterScale(y_data=-1)
scale.fit(dftrain)
dftrain = scale.apply(dftrain)
dfdev = scale.apply(dfdev)

nseries = dftrain.shape[1] - 1

Using TensorFlow backend.


In [ ]:
def submit(net_model, niter, lr, fname, exo=True):
    # Start clean
    tf.reset_default_graph()
    sess = tf.Session()
    
    if exo:
        extra = 2
    else:
        extra = 0

    # Setup placeholders for input and output
    x = tf.placeholder(tf.float32, shape=[None, present*nseries+extra], name="x")
    y = tf.placeholder(tf.float32, shape=[None, future], name="y")
    
    # Similarly, setup placeholders for dev set
    xdev = tf.placeholder(tf.float32, shape=[None, present*nseries+extra], name="xdev")
    ydev = tf.placeholder(tf.float32, shape=[None, future], name="ydev")
    
    # Setup placeholders for day of year
    day = tf.placeholder(tf.float32, shape=[None, 1], name="day")
    daydev = tf.placeholder(tf.float32, shape=[None, 1], name="daydev")
    
    # Feed forward training and dev examples
    if net_model[1] is FC:
        nn = net_model[1](net_model[2], regularizer=net_model[3])
    elif net_model[1] is CNN:
        nn = net_model[1](net_model[2], regularizer=net_model[3], height=nseries, width=present)

    yhat    = nn(x)
    yhatdev = nn(xdev)

    # Define loss in training and dev set
    with tf.name_scope("loss"):
        train_loss = tf.losses.mean_squared_error(labels=y, predictions=yhat)
        dev_loss = tf.losses.mean_squared_error(labels=ydev, predictions=yhatdev)
        tf.summary.scalar("train_loss", train_loss)
        tf.summary.scalar("dev_loss", dev_loss)

    # Minimize training loss
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(lr).minimize(train_loss)

    # Collect all summaries for TensorBoard
    summ = tf.summary.merge_all()

    # Start of execution
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + fname)
    writer.add_graph(sess.graph)

    for i in range(niter):
        # create batch
        X, Y = pp.make_batch(dftrain, 100, present, future, exo=exo)
        Xdev, Ydev = pp.make_batch(dfdev, 100, present, future, exo=exo)
        D = np.random.normal(size=(100,1))    # placeholder
        Ddev = np.random.normal(size=(100,1)) # placeholder
        
        if i % 5 == 0:
            [tloss, dloss, s] = sess.run([train_loss, dev_loss, summ],
                                         feed_dict={x: X, y: Y, day: D,
                                                    xdev: Xdev, ydev: Ydev, daydev: Ddev})
            writer.add_summary(s, i)
            writer.flush()
        
        sess.run(train_step, feed_dict={x: X, y: Y, day: D,
                                        xdev: Xdev, ydev: Ydev, daydev: Ddev})

In [ ]:
net_models = [
    ('FC2', FC, [512,512,future], None),
    
    ('FC2-l1', FC, [512,512,future], l1()),

    ('FC2-l2', FC, [512,512,future], l2())
]

def main():
    for lr in [1e-2]:
        for net_model in net_models:
            for exo in [True, False]:
                niter = 1500
                fname = "{},lr={},exo={}".format(net_model[0], lr, exo)

                submit(net_model, niter, lr, fname, exo)
    
main()